In [1]:
import ROOT as RT
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, DelphesSchema
from pprint import pprint
import numpy as np
import collections
import numba as nb

Welcome to JupyROOT 6.24/06


In [2]:
PIDof = {'LLP': 1500001, 'muon' : 13, 'Bmeson': 500} #Bmesons are only found in the 500s so I'm just picking the first and making a range lager in code

def getEvents(fileURL, treename, start, stop):
    return NanoEventsFactory.from_root(fileURL, treepath = treename, schemaclass = DelphesSchema, entry_start = start, entry_stop = stop).events().Particle

#to make dictionary-like objects easier
def makehash():
    return collections.defaultdict(makehash)

def loader(filedir, filenames, treename, start, stop):
    ctauSamples = {}
    for key in filenames:
        ctauSamples[key] = getEvents(filedir + filenames[key], treename, start, stop)

    return ctauSamples

def getLLPs(particle): 
    llp = particle[particle.PID == PIDof['LLP']]
    
    '''
    r = np.sqrt(llp.decayX**2 + llp.decayY**2) 
    trueIn = ((abs(llp.Eta) < 2.4) &
              (abs(llp.decayZ) > 6500) & (abs(llp.decayZ) < 11000) &
              (r < 6955)) 
    llp = llp[trueIn]'''
    
    statusMask = llp.Status == 91
    llp = llp[statusMask]
    
    aloneMask = ak.count(ak.flatten(llp.parent.children.PID, axis = -1), axis = -1) == 2
    llp = llp[aloneMask] 
    return llp

def getChildren(particle):
    return particle.children

def getSiblings(particle):
    iD = particle.fUniqueID
    siblings = particle.parent.children
    siblings = siblings[siblings.fUniqueID != iD]
    siblings = siblings[siblings.PID != PIDof['LLP']]
    return siblings

def getParents(particle):
    return particle.parent

def getPiblings(particle):
    iD = particle.parent.fUniqueID
    piblings = particle.parent.parent.children
    piblings = piblings[piblings.fUniqueID != iD]
    return piblings
    
def getCousins(particle):
    piblings = getPiblings(particle)
    cousins = piblings.children
    return cousins
    
def genParticleDict(particle):
    llps = getLLPs(particle)
    siblings = getSiblings(llps)
    parents = getParents(llps)

    particleDict = {
        'llps' : llps,
        'kaons' : siblings,
        'Bmesons' : parents,
    }
    return particleDict

def deltaR(particle1, particle2):
    deltaPhi = particle1.Phi - particle2.Phi
    deltaEta = particle1.Eta - particle2.Eta
    return np.sqrt(deltaPhi**2 + deltaEta**2)

In [3]:
address = 'root://cmsxrootd.fnal.gov/'
#address = 'root://eospublic.cern.ch/'
#address = 'root://cmseos.fnal.gov/'
#address = 'root://cms-xrd-global.cern.ch/'


filedir   = address + "/store/user/aaportel/hadd/"
filenames = {'1e1mm': "m1_ct10mm.root",
             '1e2mm': "m1_ct100mm.root",
             '1e3mm': "m1_ct1000mm.root",
             '1e4mm': "m1_ct10000mm.root"}
treename  = "Delphes"

# initialize dictionaries
hists = makehash()
for ctau in filenames:
    hists[ctau]['Kaon dR'] = RT.TH1F('Delta R', f'Kaon-LLP dR, ctau = {ctau}', 60, 0, 2)
    
    hists[ctau]['Ellp'] = RT.TH1F('Energy (GeV)', f'LLP Energy, ctau = {ctau}', 30, 0, 3000)
    hists[ctau]['Ekaon'] = RT.TH1F('Energy (GeV)', f'Kaon Energy, ctau = {ctau}', 30, 0, 3000)
    hists[ctau]['EBmeson'] = RT.TH1F('Energy (GeV)', f'Bmeson Energy, ctau = {ctau}', 30, 0, 3000)
    
    hists[ctau]['Ptllp'] = RT.TH1F('P_{T} (GeV)', f'LLP PT, ctau = {ctau}', 30, 0, 1600)
    hists[ctau]['Ptkaon'] = RT.TH1F('P_{T} (GeV)', f'Kaon PT, ctau = {ctau}', 30, 0, 1600)
    hists[ctau]['PtBmeson'] = RT.TH1F('P_{T} (GeV)', f'Bmeson PT, ctau = {ctau}', 30, 0, 1600)
    
for ctau in hists:
    for name in hists[ctau]:
        hists[ctau][name].Sumw2()
        hists[ctau][name].SetDirectory(0)

Warning in <TROOT::Append>: Replacing existing TH1: Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: P_{T} (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: P_{T} (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Delta R (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: P_{T} (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: P_{T} (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: P_{T} (GeV) (Potential memory leak).
Warning in <TROOT::A

In [4]:
def mainloop(hists):        
    start = 0
    stop = 10000
    step = 100
    for i in range(start, stop, step):
        print(f'{i/(stop-start)*100}% done \r')
        ctauSamples = loader(filedir, filenames, treename, i, i+step)
        particleDict = {}
        for ctau in filenames:
            particleDict[ctau] = genParticleDict(ctauSamples[ctau])

        for ctau in filenames:
            llps = particleDict[ctau]['llps']
            kaons = particleDict[ctau]['kaons']
            Bmesons = particleDict[ctau]['Bmesons']

            for r in ak.flatten(deltaR(llps, kaons), axis = None):
                hists[ctau]['Kaon dR'].Fill(r)
                
            for e in ak.flatten(llps.E, axis = None):
                hists[ctau]['Ellp'].Fill(e)
                
            for e in ak.flatten(kaons.E, axis = None):
                hists[ctau]['Ekaon'].Fill(e)
                
            for e in ak.flatten(Bmesons.E, axis = None):
                hists[ctau]['EBmeson'].Fill(e)
                
            for pt in ak.flatten(llps.PT, axis = None):
                hists[ctau]['Ptllp'].Fill(pt)
                
            for pt in ak.flatten(kaons.PT, axis = None):
                hists[ctau]['Ptkaon'].Fill(pt)
                
            for pt in ak.flatten(Bmesons.PT, axis = None):
                hists[ctau]['PtBmeson'].Fill(pt)
                
    return particleDict

particleDict = mainloop(hists)

0.0% done 
1.0% done 
2.0% done 
3.0% done 
4.0% done 
5.0% done 
6.0% done 
7.000000000000001% done 
8.0% done 
9.0% done 
10.0% done 
11.0% done 
12.0% done 
13.0% done 
14.000000000000002% done 
15.0% done 
16.0% done 
17.0% done 
18.0% done 
19.0% done 
20.0% done 
21.0% done 
22.0% done 
23.0% done 
24.0% done 
25.0% done 
26.0% done 
27.0% done 
28.000000000000004% done 
28.999999999999996% done 
30.0% done 
31.0% done 
32.0% done 
33.0% done 
34.0% done 
35.0% done 
36.0% done 
37.0% done 
38.0% done 
39.0% done 
40.0% done 
41.0% done 
42.0% done 
43.0% done 
44.0% done 
45.0% done 
46.0% done 
47.0% done 
48.0% done 
49.0% done 
50.0% done 
51.0% done 
52.0% done 
53.0% done 
54.0% done 
55.00000000000001% done 
56.00000000000001% done 
56.99999999999999% done 
57.99999999999999% done 
59.0% done 
60.0% done 
61.0% done 
62.0% done 
63.0% done 
64.0% done 
65.0% done 
66.0% done 
67.0% done 
68.0% done 
69.0% done 
70.0% done 
71.0% done 
72.0% done 
73.0% done 
74.0% done 
75

In [5]:
colorPalette = [RT.kRed, RT.kBlue, RT.kOrange, RT.kGreen+3]
interestQuantities = ['Kaon dR','Ellp','Ekaon','EBmeson','Ptllp','Ptkaon','PtBmeson']
normhists = makehash()
for count, ctau in enumerate(filenames):
    for name in interestQuantities:
        #fit = RT.TF1(name, 'landau')
        #fit.SetLineColor(colorPalette[count])
        tot = hists[ctau][name].Integral()
        normhists[name][ctau] = (hists[ctau][name].Clone())
        normhists[name][ctau].Scale(1/tot)
        #normhists[name][ctau].Fit(fit)
        #hists[ctau][name].Fit(fit, 'E')


In [6]:
colorPalette = [RT.kRed, RT.kBlue, RT.kOrange, RT.kGreen+3]
shapePalette = [RT.kFullCircle, RT.kFullSquare, RT.kFullTriangleUp, RT.kFullTriangleDown]

reorganizedHists = makehash()
for ctau in hists:
    for key in hists[ctau]:
        reorganizedHists[key][ctau] = hists[ctau][key]

EhadFileName = 'PtEhists_nocut.pdf'
a = RT.TCanvas()

a.SetLogy()
a.Print(EhadFileName + "[")


for key in reorganizedHists:
    leg = RT.TLegend(.75,.75,.95,.95)
    for count, ctau in enumerate(reorganizedHists[key]):
        RT.gStyle.SetOptStat(0)
        reorganizedHists[key][ctau].SetTitle(key)
        reorganizedHists[key][ctau].SetMarkerStyle(shapePalette[count])
        reorganizedHists[key][ctau].SetMarkerColor(colorPalette[count])
        reorganizedHists[key][ctau].SetLineColor(colorPalette[count])
        leg.AddEntry(reorganizedHists[key][ctau], ctau)

        reorganizedHists[key][ctau].GetYaxis().SetTitle('Counts')
        reorganizedHists[key][ctau].SetAxisRange(.5,1.5e3,"Y")
        if key == 'Kaon dR':
            reorganizedHists[key][ctau].GetXaxis().SetTitle('Kaon-LLP \Delta R')
            reorganizedHists[key][ctau].SetTitle('Kaon-LLP \DeltaR')
        else:
            reorganizedHists[key][ctau].GetXaxis().SetTitle('Energy (GeV)')
            reorganizedHists[key][ctau].SetTitle(key)
            
        if count == 0:
            reorganizedHists[key][ctau].Draw('X0 E')
        else:
            reorganizedHists[key][ctau].Draw('X0 E same')
    
    leg.Draw()
    a.Print(EhadFileName)
    a.Clear()       
    
a.Print(EhadFileName + "]") #closes pdf
a.Close() #closes canvas

EhadFileName = 'norm_PtEhists_nocut.pdf'
b = RT.TCanvas()

b.SetLogy()
b.Print(EhadFileName + "[")

for key in normhists:
    leg = RT.TLegend(.75,.75,.95,.95)
    for count, ctau in enumerate(normhists[key]):
        RT.gStyle.SetOptStat(0)
        normhists[key][ctau].SetMarkerStyle(shapePalette[count])
        normhists[key][ctau].SetMarkerColor(colorPalette[count])
        normhists[key][ctau].SetLineColor(colorPalette[count])
        leg.AddEntry(normhists[key][ctau], ctau)

        normhists[key][ctau].GetYaxis().SetTitle('Counts')
        normhists[key][ctau].SetAxisRange(.75e-3,1.5,"Y")
        if key == 'Kaon dR':
            normhists[key][ctau].GetXaxis().SetTitle('Kaon-LLP \Delta R')
            normhists[key][ctau].SetTitle('Kaon-LLP \DeltaR')
        else:
            normhists[key][ctau].GetXaxis().SetTitle('Energy (GeV)')
            normhists[key][ctau].SetTitle(key)

            
        if count == 0:
            normhists[key][ctau].Draw('X0 E')
        else:
            normhists[key][ctau].Draw('X0 E same')
        
    leg.Draw()
    b.Print(EhadFileName)
    b.Clear()           
        
    
b.Print(EhadFileName + "]") #closes pdf
b.Close() #closes canvas

Info in <TCanvas::Print>: pdf file PtEhists_nocut.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file PtEhists_nocut.pdf
Info in <TCanvas::Print>: pdf file PtEhists_nocut.pdf has been closed
Info in <TCanvas::Print>: pdf file norm_PtEhists_nocut.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file norm_PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file norm_PtEhists_nocut.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file norm_PtEhist

#### This is the process that might produce some LLPs
<img src=images/llpProcess.png> 

#### We want to make some collections of the extended family

In [7]:
for key in reorganizedHists:
    for ctau in reorganizedHists[key]:
        total = reorganizedHists[key][ctau].Integral()
        print(f'{key}, {ctau}: {total}')

Kaon dR, 1e1mm: 8043.0
Kaon dR, 1e2mm: 7149.0
Kaon dR, 1e3mm: 6112.0
Kaon dR, 1e4mm: 5060.0
Ellp, 1e1mm: 8080.0
Ellp, 1e2mm: 7250.0
Ellp, 1e3mm: 6283.0
Ellp, 1e4mm: 5410.0
Ekaon, 1e1mm: 8073.0
Ekaon, 1e2mm: 7220.0
Ekaon, 1e3mm: 6235.0
Ekaon, 1e4mm: 5270.0
EBmeson, 1e1mm: 8076.0
EBmeson, 1e2mm: 7249.0
EBmeson, 1e3mm: 6282.0
EBmeson, 1e4mm: 5410.0
Ptllp, 1e1mm: 8069.0
Ptllp, 1e2mm: 7247.0
Ptllp, 1e3mm: 6283.0
Ptllp, 1e4mm: 5410.0
Ptkaon, 1e1mm: 8065.0
Ptkaon, 1e2mm: 7212.0
Ptkaon, 1e3mm: 6235.0
Ptkaon, 1e4mm: 5270.0
PtBmeson, 1e1mm: 7970.0
PtBmeson, 1e2mm: 7219.0
PtBmeson, 1e3mm: 6278.0
PtBmeson, 1e4mm: 5400.0


In [8]:
ak.min(particleDict['1e1mm']['Bmesons'].PT)

56.648987